In [ ]:
cd ..

In [ ]:
from src.imports import *

from src.transformers import *
from src.vars import *

In [ ]:
train = pd.read_csv("data/source/train.csv")

In [ ]:
test = pd.read_csv("data/source/test.csv")

In [ ]:
train

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.dtypes

In [ ]:
train.dtypes.value_counts()

In [ ]:
train.describe(include=np.number).round(2)

In [ ]:
dd = train.describe(exclude=np.number).round(2)
dd

In [ ]:
train.Transported = train.Transported.astype(int)
train

In [ ]:
X = train.drop("Transported", axis=1)
y = train["Transported"]

In [ ]:
dd = dd.T
dd

In [ ]:
sns.pairplot(train, corner=True, hue="Transported")

In [ ]:
scaler_list = [
    StandardScaler(),
    "passthrough",
    MinMaxScaler(),
    RobustScaler(),
    Normalizer(),
    QuantileTransformer(n_quantiles=100, output_distribution="uniform"),
    QuantileTransformer(n_quantiles=100),
]

In [ ]:
pipeline = Pipeline(
    [
        ("column_selector", ColumnSelector()),
        ("log_transformer", LogTransformer()),
        ("preprocessor", ColumnTransformer([])),
        ("sampler_1", RandomUnderSampler()),
        ("imputer", KNNImputer()),
        ("scaler", StandardScaler()),
        ("sampler_2", RandomUnderSampler()),
        ("estimator", LogisticRegression()),
    ]
)

In [ ]:
param_grid = {
    "log_transformer__threshold": [0.5, 1, 3, 5, 10, 7, 12],
    "preprocessor__transformers": [t0, t1],
    "sampler_1": [RandomUnderSampler(), "passthrough"],
    "scaler": scaler_list,
    "sampler_2": [RandomUnderSampler(), "passthrough"],
    "estimator": [
        LogisticRegression(),
    ],
}

In [ ]:
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,  # StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    scoring="accuracy",
    n_jobs=-1,
    return_train_score=True,
    verbose=1,
)

In [ ]:
grid.fit(X, y)

In [ ]:
def resultize(grid):
    res = grid.cv_results_
    res = pd.DataFrame(res)

    res = res.sort_values(by="rank_test_score")
    res.drop(columns="rank_test_score", inplace=True)

    cols = [i for i in res.columns if "split" not in i]

    res = res[cols]

    res = res.round(2)

    return res

In [ ]:
res = resultize(grid).head(10)
res

In [ ]:
res.params.iloc[0]

In [ ]:
res.params.iloc[1]

In [ ]:
res.params.iloc[2]

In [ ]:
param_grid = {
    "log_transformer__threshold": [3, 5, 10, 7, 12],
    "preprocessor__transformers": [t0, t1],
    "sampler_1": [RandomUnderSampler(), "passthrough"],
    "scaler": ["passthrough", StandardScaler(), Normalizer()],
    "sampler_2": ["passthrough"],
    "estimator": [
        RandomForestClassifier(),
    ],
}

In [ ]:
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,  # StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    scoring="accuracy",
    n_jobs=-1,
    return_train_score=True,
    verbose=1,
)

grid.fit(X, y)

In [ ]:
res = resultize(grid).head(10)
res

In [ ]:
grid